# Kipr personal toolbox speed checks


In [1]:
import kipr as kp
import numpy as np

## Addition

In [ ]:
shape = (10, 10, 10, 10, 10, 10)
a, b = kp.arr('randn', shape=shape), kp.arr('randn', shape=shape)
%timeit a + b

a, b = np.random.randn(*shape).astype(float32), np.random.randn(*shape).astype(float32)
%timeit a + b

TypeError: an integer is required

KarrayError: Shape must be a sequence of integers.

## Subscript

In [4]:
shape = (10, 10, 10, 10, 10, 10)
values = np.random.randn(*shape)
values2 = np.random.randn(10000, 300, 10)

a = kp.arr(values)
b = kp.arr(values2)
print('kipr time:')
%timeit a[0], a[3, 2, 5, 2, 5, 2], a[:, ..., 1:7:2], a[[3, 2, 5, 2, 5, 2], :, 0], b[3, 2, 5], b[:, ..., 1:7:2]

a = values
b = values2
print('numpy time:')
%timeit a[0].copy(), a[3, 2, 5, 2, 5, 2].copy(), a[:, ..., 1:7:2].copy(), a[[3, 2, 5, 2, 5, 2], :, 0], b[3, 2, 5].copy(), b[:, ..., 1:7:2]

kipr time:
167 ms ± 30.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
numpy time:
2.03 ms ± 70.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Loading from numpy array

In [5]:
shape = (10, 10, 10, 10, 10)
a = np.random.rand(*shape)
b = a.astype(np.float32)

%timeit kp.arr(a)
%timeit kp.arr(b)

8.18 ms ± 156 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
160 µs ± 1.04 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## ReLU

In [2]:
shape = (10, 10, 10, 10, 10, 10)
a = kp.arr('random', shape=shape)
%timeit kp.relu(a)

a = np.random.rand(*shape).astype(np.float32)
%timeit a * (a > 0)

1.27 ms ± 5.86 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
4.5 ms ± 76.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
